In [88]:
# -*- coding: utf-8 -*-
import torch
import math
import matplotlib.pyplot as plt
import numpy as np
from mpmath import mp
import scipy.io as scio

dtype = torch.float
device = torch.device("cuda:0")

weights = scio.loadmat("testWeights.mat")
pw1 = torch.tensor(weights["w1"], dtype=torch.float32)
pb1 = torch.squeeze(torch.tensor(weights["b1"], dtype=torch.float32))
pw2 = torch.tensor(weights["w2"], dtype=torch.float32)
pb2 = torch.squeeze(torch.tensor(weights["b2"], dtype=torch.float32))

print(pw1)

tensor([[-279.8623],
        [-280.0815],
        [ 280.0021],
        [-279.9912],
        [ 280.0005],
        [-279.9974],
        [ 280.0035],
        [-280.0072],
        [ 279.9928],
        [-279.9911],
        [-279.9731],
        [ 279.9898],
        [-279.9708],
        [-280.0371],
        [ 279.9823],
        [ 279.9776],
        [ 280.0908],
        [-280.0201],
        [-279.9217],
        [ 279.9635],
        [ 279.9633],
        [-280.0237],
        [ 279.9846],
        [ 279.9956],
        [ 280.0270],
        [ 280.2838],
        [-280.0073],
        [ 279.9891],
        [-279.9911],
        [-280.0179],
        [ 279.9959],
        [ 279.9726],
        [-280.0144],
        [-280.0029],
        [-280.0018],
        [ 279.9992],
        [-279.9991],
        [ 280.0262],
        [ 279.9951],
        [ 279.9878],
        [ 280.0055],
        [-279.9905],
        [ 280.0023],
        [-279.9997],
        [ 279.9987],
        [-280.0033],
        [ 280.0002],
        [-280

In [89]:
class Poly1(torch.nn.Module):
    def __init__(self,dimension):
        super().__init__()
        self.dim = dimension

    def forward(self, x):
        # return poly(1, self.dim)
        return -torch.log(1 - (-torch.exp(x)))

class Qnet(torch.nn.Module):
    def __init__(self, dimension):
        super().__init__()
        with torch.no_grad():
            self.model = torch.nn.Sequential(torch.nn.Linear(dimension+1, 2**dimension, bias=False, dtype=torch.float32, device=device), 
            Poly1(dimension), 
            torch.nn.Linear(2**dimension, 1, bias=False, dtype=torch.float32,device=device))

        tpp = []
        for i in np.arange(0, 2**dimension, 1):
            tpp.append(list(np.binary_repr(i, width=dimension)))
        
        tpp = np.array(tpp).astype(float)
        S = torch.tensor(np.concatenate(((tpp == 0) * -1 + tpp, np.full((2**dimension, 1), -1)), axis=1), dtype=torch.float32, device=device)
        w3 = (-torch.prod(S, axis=1).T).reshape(1,S.size(0))

        # S[0][0] = -0.75
        # S[1][0] = 0

        self.model[0].weight = torch.nn.Parameter(S)
        self.model[2].weight = torch.nn.Parameter(w3)

    def forward(self, x):
        return self.model(x)

class Intergrator1():
    def __init__(self):
        self.qnet = Qnet(1)

    def apply(self, W1, B1, W2, B2, start, end):
        self.qnet.model[0].weight[0][0] = -end
        self.qnet.model[0].weight[1][0] = -start
        y = torch.cat((W1, B1.reshape(W1.size(0),1)), axis=1)
        res = torch.div(self.qnet(y), torch.prod(W1, axis = 1).reshape(W1.size(0),1)) + end-start
        self.model = torch.nn.Linear(W2.size(1),1, dtype=torch.float32,device=device)
        self.model.weight = torch.nn.Parameter(W2)
        self.model.bias = torch.nn.Parameter(B2*(end-start))#2**self.dim))

        return self.model(res.T)

In [90]:
W1 = torch.tensor([[-53.099407, 1.5240264, -0.17084326,-1.380339, -0.72163534, -0.30636203, -0.7825347, 2.1130767, 2.3055365, 3.301027]], dtype=torch.float32, device=device)
W1 = W1.T
B1 = torch.tensor([26.595804, -1.7131307, 0.44687968, -0.87286687, -3.1546798, -2.0160902, -0.24203205, 0.514264, -0.19717565, 4.175638], dtype=torch.float32, device=device)

W2 = torch.tensor([[-0.30055588, 3.15199, 1.475628, -0.20895372, 0.3831912, 3.4266005, 0.19389513, -2.9015236, 1.8864253, -2.0440385]], dtype=torch.float32, device=device)
B2 = torch.tensor([[1.4603579]], dtype=torch.float32, device=device)

In [92]:
test = Intergrator1()
with torch.no_grad():
    tests = test.apply(W1,B1, W2, B2, 0, 0.75)

print(test.qnet.model[0].weight)
print(test.qnet.model[0].bias)
print(test.qnet.model[2].weight)
print(test.qnet.model[2].bias)

print(tests)

Parameter containing:
tensor([[-0.7500, -1.0000],
        [ 0.0000, -1.0000]], device='cuda:0', requires_grad=True)
None
Parameter containing:
tensor([[-1.,  1.]], device='cuda:0', requires_grad=True)
None
tensor([[0.1980]], device='cuda:0')
